In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta
import pickle
import requests
import numpy as np
from glicko2 import Glicko2
from trueskill import TrueSkill
from scipy.stats import norm
from collections import defaultdict

## First look at the data

This dataset contains results from every Bundesliga match from 1993-1994 to 2021-2022. It also includes half time results, but only since 1995-96. Columns include Division (denoted as D1), HomeTeam, AwayTeam, FTHG (final time home goals), FTAG (final time away goals), FTR (full time result), HTHG (half time home goals), HTAG (half time away goals), HTR (half time result), and season.

Data compiled into one file from this site: http://www.football-data.co.uk/germanym.php

In [3]:
df = pd.read_csv('Bundesliga_Results.csv', parse_dates=['Date'])

In [4]:
# some basic cleaning and rewriting
df.sort_values(by='Date', inplace=True)
df.reset_index(inplace=True)
# clean the team names
df.HomeTeam = df.HomeTeam.apply(lambda x: x.lower().replace(' ', ''))
df.AwayTeam = df.AwayTeam.apply(lambda x: x.lower().replace(' ', ''))

# deduct winner and loser from the scores
# add draws to the winner, doesn't really matter
df['home_win'] = np.where(df.FTHG >= df.FTAG, 1, 0)
df['draw'] = np.where(df.FTHG==df.FTAG, 1, 0)
df['away_win'] = np.where(df.FTHG < df.FTAG, 1, 0)

df['winner'] = (df.HomeTeam * (df.home_win) + df.AwayTeam * df.away_win)
df['loser'] = (df.HomeTeam * (1 - df.home_win) + df.AwayTeam * (1 - df.away_win))

print('Dataset contains {} matches from {} to {}. Home won {} ({}%). There were {} draws ({}%).'.format(len(df), df.Date.min().date(), df.Date.max().date(), df.home_win.sum(), int(100*df.home_win.sum()/len(df)), df.draw.sum(), int(100*df.draw.sum()/len(df)) ))

Dataset contains 10098 matches from 1993-01-09 to 2022-12-03. Home won 7207 (71%). There were 2554 draws (25%).


In [5]:
df.head()

,index,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Season,home_win,draw,away_win,winner,loser
0,44,44,D1,1993-01-09,werderbremen,duisburg,1,5,A,NaN,NaN,NaN,1993-94,0,0,1,duisburg,werderbremen
1,42,42,D1,1993-01-09,m'gladbach,schalke04,3,2,H,NaN,NaN,NaN,1993-94,1,0,0,m'gladbach,schalke04
2,41,41,D1,1993-01-09,kaiserslautern,nurnberg,3,1,H,NaN,NaN,NaN,1993-94,1,0,0,kaiserslautern,nurnberg
3,40,40,D1,1993-01-09,hamburg,leverkusen,2,1,H,NaN,NaN,NaN,1993-94,1,0,0,hamburg,leverkusen
4,39,39,D1,1993-01-09,fckoln,freiburg,2,0,H,NaN,NaN,NaN,1993-94,1,0,0,fckoln,freiburg


In [6]:
df.to_csv('data_preprocessed.csv')